In [7]:
import pandas as pd
from clingo import Control
import ast
from matching_utils import get_invalid_matches

students_df =pd.read_csv('../data/studenten.csv')
mentors_df =pd.read_csv('../data/mentoren.csv')

education_mapping = {
    'Associate': 1,
    'Bachelor': 2, 
    'Master': 3,
    'PhD': 4
}

def generate_asp_facts():
    facts = []

    # student(s0).
    # education(s0, 2).
    # expertise(s0, data_science).
    # mentor(m0).
    # education(m0, 3).
    # expertise(m0, data_science).
    
    for _, row in students_df.iterrows():
        student_id = f"s{row.name}"  
        facts.append(f"student({student_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({student_id}, {edu_level}).")
        
        subject = row['Onderwerp'].lower().replace(' ', '_')
        facts.append(f"expertise({student_id}, {subject}).")
    
    for _, row in mentors_df.iterrows():
        mentor_id = f"m{row.name}"  
        facts.append(f"mentor({mentor_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({mentor_id}, {edu_level}).")
        
        subjects = ast.literal_eval(row['Onderwerpen'])
        for subject in subjects:
            subject_clean = subject.lower().replace(' ', '_')
            facts.append(f"expertise({mentor_id}, {subject_clean}).")
    
    return "\n".join(facts)

asp_facts = generate_asp_facts()

ctl = Control()
ctl.add("base", [], f"""
{asp_facts}

% Matching constraints
valid_mentor(S, M) :-
    student(S),
    mentor(M), 
    expertise(M, Subj), 
    expertise(S, Subj), 
    education(M, EM), 
    education(S, ES), 
    EM > ES.

% Allow 0 or 1 mentor per student 
{{ match_mentor(S, M) : valid_mentor(S, M) }} <= 1 :- student(S).

% Maximize the number of matched students 
#maximize {{ 1,S : match_mentor(S,M) }}.

% Output
#show match_mentor/2.
""")

ctl.ground([("base", [])])

def make_matches():
    matches = []
    
    def collect_matches(model):
        nonlocal matches
        matches = []  # Reset for each model
        
        for symbol in model.symbols(shown=True):
            if symbol.name == "match_mentor":
                student_id = symbol.arguments[0].name
                mentor_id = symbol.arguments[1].name
                
                student_idx = int(student_id[1:])  
                mentor_idx = int(mentor_id[1:])    
                
                # Get full student data
                student_data = {
                    'voornaam': students_df.iloc[student_idx]['Voornaam'],
                    'achternaam': students_df.iloc[student_idx]['Achternaam'],
                    'opleidingsniveau': students_df.iloc[student_idx]['Opleidingsniveau'],
                    'onderwerp': students_df.iloc[student_idx]['Onderwerp']
                }
                
                # Get full mentor data
                mentor_data = {
                    'voornaam': mentors_df.iloc[mentor_idx]['Voornaam'],
                    'achternaam': mentors_df.iloc[mentor_idx]['Achternaam'],
                    'opleidingsniveau': mentors_df.iloc[mentor_idx]['Opleidingsniveau'],
                    'onderwerpen': ast.literal_eval(mentors_df.iloc[mentor_idx]['Onderwerpen'])
                }
                
                matches.append((student_data, mentor_data))
    
    result = ctl.solve(on_model=collect_matches)
    
    if result.satisfiable:
        return matches
    else:
        return []

# Usage
matches = make_matches()
matches
get_invalid_matches(matches)
# for student, mentor in matches:
#     print(f"{student['voornaam']} {student['achternaam']} ({student['opleidingsniveau']}, {student['onderwerp']}) → {mentor['voornaam']} {mentor['achternaam']} ({mentor['opleidingsniveau']}, {mentor['onderwerpen']})")

[]